# Merger bushes
Script to explore properties of merger trees for which there may be more than one descendant halo.
Note that precludes the use of simple pointers such as nextSibling to search over past trees - because more than one descendant may share the same halo.  Instead, as in the output from ahf_MULTIDVIR, each halo must keep its own list of progenitors.

## Preliminaries
This section reads in the data from the ahf_MULTIDVIR output file
and does some simple sanity checks.

In [1]:
# Imports.
import h5py
import numpy as np

In [2]:
# Parameters.
nSnap=62

In [3]:
# Open HDF5 file, if not already loaded
try:
    fin
except:
    fin=h5py.File('/Users/petert/data/Sussing/network.hdf5','r')

In [4]:
# Read in existing data about halos
HalosInSnap=fin['HaloCatalogue_0/HalosInSnap']
nHaloSnap=HalosInSnap[:]
nHalo=sum(nHaloSnap)

In [6]:
# Read in existing data about Descendants
NHalosDescendant=fin['Descendant_0/NHalosDescendant']
nDesc=NHalosDescendant[:] # Number of descendants for each halo.
HalosDescendant=fin['Descendant_0/HalosDescendant']
haloID=HalosDescendant['haloID']
share=HalosDescendant['share']
# Sanity checks
assert nHalo == len(nDesc)
assert sum(nDesc) == len(haloID)

In [7]:
# ahf_MULTIDVIR labels halos from 0 within each snap.
# To identify the location within the whole array, 
# we need to know the offsets of each snap.
# nHaloSnapCum provides the cumulative halo count needed to do that.
nHaloSnapCum=np.zeros(nSnap,dtype=np.int32)
nHaloSnapCum[0]=nHaloSnap[0]
for iSnap in range(1,nSnap): 
    nHaloSnapCum[iSnap]=nHaloSnapCum[iSnap-1]+nHaloSnap[iSnap]
assert nHaloSnapCum[nSnap-1] == nHalo

## Meaning of 'share' 
I want to test what "share" means.  I presume that it is the number
of particles that a halo and its descendant have in common.
This section tests that idea.